# Speed optimization and weather dataframes

In [1]:
import os
import re
import json
import math
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

/Users/andersvandvik/Repositories/masters-thesis/venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## General parameters

In [2]:
PROJECT_PATH = os.path.dirname(os.path.abspath('.'))
NSO_PATH = f'{PROJECT_PATH}/output/solstorm/alns/managerial/speed_opt/nso/'
SO_PATH = f'{PROJECT_PATH}/output/solstorm/alns/managerial/speed_opt/so/'

DISC_PARAM = 4
WORST_WEATHER_STATE = 3
SPEED_IMPACTS = [0.0, 0.0, 2.0, 3.0]
SERVICE_IMPACTS = [1.0, 1.2, 1.3, 2.0]
DESIGN_SPEED = 12.0
FC_IDLING = 120
FC_SERVICING = 170

INSTANCE_KEY = 'instance'
NSO_P_KEY = 'nso (p)'
SO_P_KEY = 'so (p)'
NSO_M_KEY = 'nso (m)'
SO_M_KEY = 'so (m)'
NSO_C_KEY = 'nso (c)'
SO_C_KEY = 'so (c)'
RED_FC_P_KEY = 'cost red (p)'
RED_FC_M_KEY = 'cost red (m)'
RED_FC_C_KEY = 'cost red (c)'
RED_EM_P_KEY = 'em red (p)'
RED_EM_M_KEY = 'em red (m)'
RED_EM_C_KEY = 'em red (c)'

PERFECT_KEY = 'perfect'
MIXED_KEY = 'mixed'
CRITICAL_KEY = 'critical'

BEST_OBJ_KEY = 'best_objective'

VOYAGES_KEY = 'voyages'
SEQUENCE_KEY = 'sequence'
TIME_POINTS_KEY = 'time_points'
SPEED_KEY = 'speed'
ORDERS_KEY = 'orders'
INSTALLATION_KEY = 'installation'
LAT_KEY = 'latitude'
LON_KEY = 'longitude'
SCENARIOS_KEY = 'scenarios'
WS_KEY = 'weather_scenario'
END_DEPOT_KEY = 'ED'
ARR_TIME_KEY = 'arrival_time'
SERVICE_TIME_KEY = 'service_time'
END_TIME_KEY = 'end_time'
FLEET_KEY = 'fleet'
FC_DESIGN_SPEED_KEY = 'fc_design_speed'

## Functions

In [3]:
def get_weather_forecast(weather_scenario):
    weather_path = f'{PROJECT_PATH}/src/main/resources/constant/weather.json'
    with open(weather_path) as file:
        weather_json = json.load(file)
    weather_forecast = weather_json[SCENARIOS_KEY][weather_scenario]
    wf = [weather_forecast[math.floor(i / DISC_PARAM)] for i in range(len(weather_forecast) * DISC_PARAM)]
    return wf

def get_latlon_list():
    installations_path = f'{PROJECT_PATH}/src/main/resources/constant/installations.json'
    with open(installations_path) as file:
        installations_json = json.load(file)
    latlon_list = []
    for inst in installations_json:
        latlon_list.append((installations_json[inst][LAT_KEY], installations_json[inst][LON_KEY]))
    return latlon_list

def get_instance_json(solution_json):
    instance_file = solution_json[INSTANCE_KEY]
    instance_path = f'{PROJECT_PATH}/src/main/resources/managerial_insights/{instance_file}'
    with open(instance_path) as file:
        instance_json = json.load(file)
    return instance_json

def get_vessels_json():
    vessels_path = f'{PROJECT_PATH}/src/main/resources/constant/vessels.json'
    with open(vessels_path) as file:
        vessels_json = json.load(file)
    return vessels_json

In [25]:
def generate_speed_opt_df(nso_path, so_path):
    nso_obj_data = map_instance_to_ws_to_obj(nso_path)
    nso_em_data = map_instance_to_emissions(nso_path)
    so_obj_data = map_instance_to_ws_to_obj(so_path)
    so_em_data = map_instance_to_emissions(so_path)
    
    df = pd.DataFrame(columns=[INSTANCE_KEY, 
                               NSO_P_KEY, SO_P_KEY, RED_FC_P_KEY, RED_EM_P_KEY,
                               NSO_M_KEY, SO_M_KEY, RED_FC_M_KEY, RED_EM_M_KEY,
                               NSO_C_KEY, SO_C_KEY, RED_FC_C_KEY, RED_EM_C_KEY])

    for instance_name in nso_obj_data:
        perfect_obj_nso = nso_obj_data[instance_name][PERFECT_KEY]
        perfect_obj_so = so_obj_data[instance_name][PERFECT_KEY]
        perfect_fc_red = abs(perfect_obj_so - perfect_obj_nso) / perfect_obj_nso
        perfect_em_nso = nso_em_data[instance_name][PERFECT_KEY]
        perfect_em_so = so_em_data[instance_name][PERFECT_KEY]
        perfect_em_red = abs(perfect_em_so - perfect_em_nso) / perfect_em_nso
        
        mixed_obj_nso = nso_obj_data[instance_name][MIXED_KEY]
        mixed_obj_so = so_obj_data[instance_name][MIXED_KEY]
        mixed_fc_red = abs(mixed_obj_so - mixed_obj_nso) / mixed_obj_nso
        mixed_em_nso = nso_em_data[instance_name][MIXED_KEY]
        mixed_em_so = so_em_data[instance_name][MIXED_KEY]
        mixed_em_red = abs(mixed_em_so - mixed_em_nso) / mixed_em_nso
        
        critical_obj_nso = nso_obj_data[instance_name][CRITICAL_KEY]
        critical_obj_so = so_obj_data[instance_name][CRITICAL_KEY]
        critical_fc_red = abs(critical_obj_so - critical_obj_nso) / critical_obj_nso
        critical_em_nso = nso_em_data[instance_name][CRITICAL_KEY]
        critical_em_so = so_em_data[instance_name][CRITICAL_KEY]
        critical_em_red = abs(critical_em_so - critical_em_nso) / critical_em_nso
        
        row = pd.Series({INSTANCE_KEY: instance_name, 
                        NSO_P_KEY: perfect_obj_nso, SO_P_KEY: perfect_obj_so, RED_FC_P_KEY: perfect_fc_red, RED_EM_P_KEY: perfect_em_red,
                        NSO_M_KEY: mixed_obj_nso, SO_M_KEY: mixed_obj_so, RED_FC_M_KEY: mixed_fc_red, RED_EM_M_KEY: mixed_em_red,
                        NSO_C_KEY: critical_obj_nso, SO_C_KEY: critical_obj_so, RED_FC_C_KEY: critical_fc_red, RED_EM_C_KEY: critical_em_red})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by=INSTANCE_KEY,
                        key=lambda x: np.argsort(index_natsorted(df[INSTANCE_KEY])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({INSTANCE_KEY: 'Mean values', 
                          NSO_P_KEY: df[NSO_P_KEY].mean(), SO_P_KEY: df[SO_P_KEY].mean(), 
                          RED_FC_P_KEY: df[RED_FC_P_KEY].mean(), RED_EM_P_KEY: df[RED_EM_P_KEY].mean(), 
                          NSO_M_KEY: df[NSO_M_KEY].mean(), SO_M_KEY: df[SO_M_KEY].mean(), 
                          RED_FC_M_KEY: df[RED_FC_M_KEY].mean(), RED_EM_M_KEY: df[RED_EM_M_KEY].mean(),
                          NSO_C_KEY: df[NSO_C_KEY].mean(), SO_C_KEY: df[SO_C_KEY].mean(), 
                          RED_FC_C_KEY: df[RED_FC_C_KEY].mean(), RED_EM_C_KEY: df[RED_EM_C_KEY].mean()})
    
    df = df.append(mean_row, ignore_index=True)
    df = df.round(4)
    return df
    
def map_instance_to_ws_to_obj(path):
    instance_to_ws_to_data = {}
    for file_name in os.listdir(path):
        if file_name == '.DS_Store':
            continue
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        ws = split_name[1]
        solution_or_history = split_name[3]
        if solution_or_history == 'history':
            continue
        
        with open(path + file_name) as file:
            solution_json = json.load(file)
        
        obj = solution_json[BEST_OBJ_KEY]
        
        if instance_name in instance_to_ws_to_data:
            if ws in instance_to_ws_to_data[instance_name]:
                data = instance_to_ws_to_data[instance_name][ws]
                data[0] += obj
                data[1] += 1
            else:
                instance_to_ws_to_data[instance_name][ws] = [obj, 1]            
        else:
            instance_to_ws_to_data[instance_name] = {ws: [obj, 1]}
    
    instance_to_ws_to_obj = {}
    for instance_name in instance_to_ws_to_data:
        instance_to_ws_to_obj[instance_name] = {}
        for ws in instance_to_ws_to_data[instance_name]:
            data = instance_to_ws_to_data[instance_name][ws]
            instance_to_ws_to_obj[instance_name][ws] = data[0] / data[1]
            
    return instance_to_ws_to_obj

def map_instance_to_emissions(path):
    instance_to_ws_to_emissions_data = {}
    for file_name in os.listdir(path):
        if file_name == '.DS_Store':
            continue
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        ws = split_name[1]
        
        solution_or_history = split_name[3]
        if solution_or_history == 'history':
            continue
        
        with open(path + file_name) as file:
            solution_json = json.load(file)
        
        emissions = calculate_emissions(solution_json)
        
        if instance_name in instance_to_ws_to_emissions_data:
            if ws in instance_to_ws_to_emissions_data[instance_name]:
                data = instance_to_ws_to_emissions_data[instance_name][ws]
                data[0] += emissions
                data[1] += 1
            else:
                instance_to_ws_to_emissions_data[instance_name][ws] = [emissions, 1]
        else:
            instance_to_ws_to_emissions_data[instance_name] = {ws: [emissions, 1]}
    
    instance_to_ws_to_emissions = {}
    for instance_name in instance_to_ws_to_emissions_data:
        instance_to_ws_to_emissions[instance_name] = {}
        for ws in instance_to_ws_to_emissions_data[instance_name]:
            data = instance_to_ws_to_emissions_data[instance_name][ws]
            instance_to_ws_to_emissions[instance_name][ws] = data[0] / data[1]
    
    return instance_to_ws_to_emissions
        

def calculate_emissions(solution_json):
    
    total_emissions = 0
    latlon_list = get_latlon_list()
    instance_json = get_instance_json(solution_json)
    vessels_json = get_vessels_json()
    weather_scenario = instance_json[WS_KEY]
    wf = get_weather_forecast(weather_scenario)
    orders_json = instance_json[ORDERS_KEY]
    
    # TODO: REMOVE
    total_consumption = 0
    
    for psv in solution_json[VOYAGES_KEY]:
        sequence = solution_json[VOYAGES_KEY][psv][SEQUENCE_KEY]
        from_inst = 0
        prev_end_time = 63

        for order in sequence:
            to_inst = orders_json[str(order)][INSTALLATION_KEY]
            
            time_points_json = solution_json[VOYAGES_KEY][psv][TIME_POINTS_KEY]
            if not bool(time_points_json):
                continue
            speed = time_points_json[str(order)][SPEED_KEY]
            
            arr_time = time_points_json[str(order)][ARR_TIME_KEY]
            service_time = time_points_json[str(order)][SERVICE_TIME_KEY]
            end_time = time_points_json[str(order)][END_TIME_KEY]
            
            from_inst_latlon = latlon_list[from_inst]
            to_inst_latlon = latlon_list[to_inst]
            distance = calculate_distance(from_inst_latlon[0], to_inst_latlon[0], from_inst_latlon[1], to_inst_latlon[1])
            
            fc_design_speed = vessels_json[FLEET_KEY][psv][FC_DESIGN_SPEED_KEY]
            
            fc_sailing = calculate_total_fc_sailing(prev_end_time, arr_time, speed, distance, fc_design_speed, wf)
            em_sailing = fc_sailing * 3
            fc_idling = calculate_fc_idling(arr_time, service_time, wf)
            em_idling = fc_idling * 3
            fc_servicing = calculate_fc_servicing(service_time, end_time, wf)
            em_servicing = fc_servicing * 3
            
            # TODO: REMOVE
            total_consumption += (fc_sailing + fc_idling + fc_servicing)
            
            total_emissions += em_sailing + em_idling + em_servicing
            
            from_inst = to_inst
            prev_end_time = end_time
        
        to_inst = 0
        time_points_json = solution_json[VOYAGES_KEY][psv][TIME_POINTS_KEY]
        if not bool(time_points_json):
            continue
        speed = time_points_json[END_DEPOT_KEY][SPEED_KEY]
        
        arr_time = time_points_json[str(order)][ARR_TIME_KEY]
        service_time = time_points_json[str(order)][SERVICE_TIME_KEY]
        end_time = time_points_json[str(order)][END_TIME_KEY]
        
        from_inst_latlon = latlon_list[from_inst]
        to_inst_latlon = latlon_list[to_inst]
        distance = calculate_distance(from_inst_latlon[0], to_inst_latlon[0], from_inst_latlon[1], to_inst_latlon[1])
        
        fc_sailing = calculate_total_fc_sailing(prev_end_time, arr_time, speed, distance, fc_design_speed, wf)
        em_sailing = fc_sailing * 3
        
        fc_idling = calculate_fc_idling(arr_time, service_time, wf)
        em_idling = fc_idling * 3
        
        fc_servicing = calculate_fc_servicing(service_time, end_time, wf)
        em_servicing = fc_servicing * 3
        
        # TODO: REMOVE
        total_consumption += (fc_sailing + fc_idling + fc_servicing)
        
        total_emissions += em_sailing + em_idling + em_servicing
        
    # TODO: REMOVE
    print(total_consumption * 0.276)
    
    return total_emissions

def calculate_distance(lat1, lat2, lon1, lon2):
    if lat1 == lat2 and lon1 == lon2:
        return 0

    theta = lon1 - lon2
    dist = math.sin(math.radians(lat1)) * math.sin(math.radians(lat2)) \
           + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(theta))

    dist = math.acos(dist)
    dist = math.degrees(dist)
    dist = dist * 60 * 1.1515
    dist = dist * 0.8684  # Nautical miles
    return dist

def calculate_total_fc_sailing(start_time, arr_time, speed, distance, fc_design_speed, wf):
    if distance == 0 or start_time == arr_time:
        return 0
    time_in_each_ws = get_time_in_each_weather_state(start_time, arr_time, wf)
    distance_in_each_ws = [speed * time_in_each_ws[ws] for ws in range(WORST_WEATHER_STATE + 1)]
    consumption = calculate_fc_sailing(distance_in_each_ws[0] + distance_in_each_ws[1], speed, 0, fc_design_speed) \
                  + calculate_fc_sailing(distance_in_each_ws[2], speed, 2, fc_design_speed) \
                  + calculate_fc_sailing(distance_in_each_ws[3], speed, 3, fc_design_speed)
    return consumption

def calculate_fc_sailing(distance, speed, weather, fc_design_speed):
    return (distance / (speed - SPEED_IMPACTS[weather])) * fc_design_speed * math.pow((speed / DESIGN_SPEED), 3)

def calculate_fc_idling(idling_start_time, idling_end_time, wf):
    time_in_each_ws = get_time_in_each_weather_state(idling_start_time, idling_end_time, wf)
    consumption = 0
    for ws in range(WORST_WEATHER_STATE + 1):
        consumption += time_in_each_ws[ws] * SERVICE_IMPACTS[ws] * FC_IDLING
    return consumption

def calculate_fc_servicing(servicing_start_time, servicing_end_time, wf):
    time_in_each_ws = get_time_in_each_weather_state(servicing_start_time, servicing_end_time, wf)
    consumption = 0
    for ws in range(WORST_WEATHER_STATE + 1):
        consumption += time_in_each_ws[ws] * SERVICE_IMPACTS[ws] * SERVICE_IMPACTS[ws] * FC_SERVICING
    return consumption

def calculate_service_time(order_size):
    if order_size == 0:  # For pickup
        return 1
    else:
        return math.ceil(order_size * 0.4)

def get_time_in_each_weather_state(start_time, end_time, wf):
    return [disc_to_exact_hours(get_time_in_weather_state(start_time, end_time, ws, wf))
            for ws in range(WORST_WEATHER_STATE + 1)]

def get_time_in_weather_state(start_time, end_time, weather_state, wf):
    curr_time = start_time
    time_spent_in_weather_state = 0
    while curr_time < end_time:
        if weather_state == wf[curr_time]:
            time_spent_in_weather_state += 1
        curr_time += 1
    return time_spent_in_weather_state

def disc_to_exact_hours(disc_time):
    return disc_time / DISC_PARAM

def hour_to_disc(hourly_time):
    return hourly_time * DISC_PARAM

def deduce_postponed_orders(solution_path):
    with open(solution_path) as file:
        solution_json = json.load(file)
    
    instance_file = solution_json[INSTANCE_KEY]
    instance_path = f'{PROJECT_PATH}/src/main/resources/managerial_insights/{instance_file}'
    with open(instance_path) as file:
        instance_json = json.load(file)
    
    all_orders = []
    for order in instance_json[ORDERS_KEY]:
        all_orders.append(int(order))
    
    for psv in solution_json[VOYAGES_KEY]:
        sequence = solution_json[VOYAGES_KEY][psv][SEQUENCE_KEY]
        for order in sequence:
            if order in all_orders:
                all_orders.remove(order)
                
    print('Postponed orders: ' + str(all_orders))
    
def find_storm_periods(weather_forecast):
    for time_point, ws in enumerate(weather_forecast):
        if ws == WORST_WEATHER_STATE:
            print(time_point, ws)
            
def print_sequences(solution_path):
    with open(solution_path) as file:
        solution_json = json.load(file)
    
    instance_file = solution_json[INSTANCE_KEY]
    instance_path = f'{PROJECT_PATH}/src/main/resources/managerial_insights/{instance_file}'
    with open(instance_path) as file:
        instance_json = json.load(file)
    
    orders_json = instance_json[ORDERS_KEY]
    
    for psv in solution_json[VOYAGES_KEY]:
        print(psv)
        sequence = solution_json[VOYAGES_KEY][psv][SEQUENCE_KEY]
        from_inst = 0
        prev_end_time = 63
        for order in sequence:
            to_inst = orders_json[str(order)][INSTALLATION_KEY]
            time_points_json = solution_json[VOYAGES_KEY][psv][TIME_POINTS_KEY]
            if not bool(time_points_json):
                continue
            arr_time = time_points_json[str(order)][ARR_TIME_KEY]
            service_time = time_points_json[str(order)][SERVICE_TIME_KEY]
            end_time = time_points_json[str(order)][END_TIME_KEY]

            print(f'  Order {order}: ({from_inst}, {prev_end_time}) -> ({to_inst}, {end_time})')

            prev_end_time = end_time
            from_inst = to_inst
        print()
    
def calculate_penalty_cost(opt_order_size, inst_id, weather_scenario):  # 0 if pickup!
    wf = get_weather_forecast(weather_scenario)
    
    installations_path = f'{PROJECT_PATH}/src/main/resources/constant/installations.json'
    with open(installations_path) as file:
        installations_json = json.load(file)
    
    latlon_list = []
    for inst in installations_json:
        latlon_list.append((installations_json[inst][LAT_KEY], installations_json[inst][LON_KEY]))
    
    latlon_depot = latlon_list[0]
    latlon_inst = latlon_list[inst_id]
    distance = calculate_distance(latlon_depot[0], latlon_inst[0], latlon_depot[1], latlon_inst[1])
    
    start_time = 63
    arr_time = start_time + math.floor(hour_to_disc(distance / DESIGN_SPEED))
    service_end_time = arr_time + calculate_service_time(opt_order_size)
    sail_cost = (calculate_total_fc_sailing(start_time, arr_time, DESIGN_SPEED, distance, 540, wf) * 0.276) * 2
    service_cost = calculate_fc_servicing(arr_time, service_end_time, wf) * 0.276

    return sail_cost + service_cost
    
def calculate_idling_cost(start_time, end_time, weather_scenario):
    wf = get_weather_forecast(weather_scenario)
    return calculate_fc_idling(start_time, end_time, wf) * 0.276

In [27]:
with open(f"{NSO_PATH}/23-27-4-1_critical_135_solution.json") as file:
    nso_json = json.load(file)
    
with open(f"{SO_PATH}/23-27-4-1_critical_917_solution.json") as file:
    so_json = json.load(file)
    
calculate_emissions(nso_json)
calculate_emissions(so_json)

print((7647.394200000002 - 5894.09231666667) / 7647.394200000002)

7647.394200000002
5894.09231666667
0.22926788360580802
Postponed orders: []


## Speed optimization dataframe

In [190]:
speedopt_df = generate_speed_opt_df(NSO_PATH, SO_PATH)
speedopt_df

,instance,nso (p),so (p),cost red (p),em red (p),nso (m),so (m),cost red (m),em red (m),nso (c),so (c),cost red (c),em red (c)
0,19-21-3-2,6541.7520,4971.4005,0.2401,0.1812,7664.4165,6062.9077,0.2090,0.1183,8634.9452,7035.5048,0.1852,0.1367
1,21-24-3-1,6871.8480,5324.1624,0.2252,0.2037,8161.4304,6595.6863,0.1918,0.1222,9135.7012,7648.1840,0.1628,0.1781
2,23-27-4-1,7463.8680,5756.6812,0.2287,0.2177,8714.5482,7052.7309,0.1907,0.1811,9679.8674,8100.4754,0.1632,0.2293
3,25-29-4-1,8802.9648,6722.5101,0.2363,0.2431,10340.8037,8277.9299,0.1995,0.1641,11388.2913,9445.6276,0.1706,0.2153
4,27-32-5-1,11225.3616,8709.7857,0.2241,0.1976,13326.5689,10807.5429,0.1890,0.1011,14790.4117,12308.4632,0.1678,0.1757
5,Mean values,8181.1589,6296.9080,0.2309,0.2087,9641.5535,7759.3595,0.1960,0.1374,10725.8434,8907.6510,0.1699,0.1870


## Weather dataframe

In [267]:
FILE_NAME = '19-21-3-2_perfect_772_solution.json'
FILE_PATH = f'{PROJECT_PATH}/output/solstorm/alns/managerial/weather/{FILE_NAME}'
deduce_postponed_orders(FILE_PATH)

Postponed orders: []


In [268]:
print_sequences(FILE_PATH)

PSV_1
  Order 0: (0, 63) -> (5, 86)
  Order 3: (5, 86) -> (15, 112)
  Order 13: (15, 112) -> (26, 118)
  Order 15: (26, 118) -> (24, 119)
  Order 6: (24, 119) -> (23, 134)
  Order 5: (23, 134) -> (21, 140)
  Order 17: (21, 140) -> (22, 141)
  Order 18: (22, 141) -> (16, 149)
  Order 2: (16, 149) -> (19, 163)
  Order 16: (19, 163) -> (20, 164)

PSV_2
  Order 8: (0, 63) -> (1, 91)

PSV_3
  Order 14: (0, 63) -> (14, 94)
  Order 4: (14, 94) -> (8, 117)
  Order 20: (8, 117) -> (8, 118)
  Order 12: (8, 118) -> (12, 127)
  Order 7: (12, 127) -> (2, 140)
  Order 10: (2, 140) -> (4, 147)
  Order 19: (4, 147) -> (4, 148)
  Order 1: (4, 148) -> (6, 156)
  Order 11: (6, 156) -> (3, 158)
  Order 9: (3, 158) -> (27, 166)

SPOT



In [266]:
calculate_penalty_cost(0, 2, 'critical')

836.6112

In [233]:
calculate_idling_cost(152, 320, 'critical')

2782.0800000000004

In [230]:
WEATHER_SCENARIO = 'critical'
weather_path = f'{PROJECT_PATH}/src/main/resources/constant/weather.json'
with open(weather_path) as file:
    weather_json = json.load(file)
weather_forecast = weather_json[SCENARIOS_KEY][WEATHER_SCENARIO]
wf = [weather_forecast[math.floor(i / DISC_PARAM)] for i in range(len(weather_forecast) * DISC_PARAM)]
find_storm_periods(wf)

152 3
153 3
154 3
155 3
156 3
157 3
158 3
159 3
160 3
161 3
162 3
163 3
164 3
165 3
166 3
167 3
168 3
169 3
170 3
171 3
172 3
173 3
174 3
175 3
176 3
177 3
178 3
179 3
180 3
181 3
182 3
183 3
184 3
185 3
186 3
187 3
188 3
189 3
190 3
191 3
192 3
193 3
194 3
195 3
196 3
197 3
198 3
199 3
200 3
201 3
202 3
203 3
204 3
205 3
206 3
207 3
208 3
209 3
210 3
211 3
212 3
213 3
214 3
215 3
216 3
217 3
218 3
219 3
220 3
221 3
222 3
223 3
224 3
225 3
226 3
227 3
228 3
229 3
230 3
231 3
232 3
233 3
234 3
235 3
236 3
237 3
238 3
239 3
240 3
241 3
242 3
243 3
244 3
245 3
246 3
247 3
248 3
249 3
250 3
251 3
252 3
253 3
254 3
255 3
256 3
257 3
258 3
259 3
260 3
261 3
262 3
263 3
264 3
265 3
266 3
267 3
268 3
269 3
270 3
271 3
272 3
273 3
274 3
275 3
276 3
277 3
278 3
279 3
280 3
281 3
282 3
283 3
284 3
285 3
286 3
287 3
288 3
289 3
290 3
291 3
292 3
293 3
294 3
295 3
296 3
297 3
298 3
299 3
300 3
301 3
302 3
303 3
304 3
305 3
306 3
307 3
308 3
309 3
310 3
311 3
312 3
313 3
314 3
315 3
316 3
317 3
318 